# PA004 - HEALTH INSURANCE CROSS-SELL

## 0- IMPORTS

In [1]:
import os

import pandas          as pd
import numpy           as np
import pandas.io.sql   as psql
import psycopg2        as pg
import sweetviz        as sv

/home/reng/anaconda3/envs/health_insurance/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 0.1 - Datas

In [ ]:
# CONECTANDO NO BANCO DE DADOS

# Credentials
host="comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com"
database="comunidadedsdb"
port=5432
user="member"
password="cdspa"
    
# Postgresql Connetion
conn = pg.connect( host=host,
                   database=database,              
                   port=5432,
                   user=user,
                   password=password
                 )

# Criando cursor
cur = conn.cursor()

#### Schema Query

In [ ]:
# VERIFICANDO SCHEMAS DO BANCO DE DADOS
query_schema = """
    SELECT nspname
    FROM pg_catalog.pg_namespace
"""
cur.execute( query_schema )

record = cur.fetchall()
print(record)

#### Table Query

In [ ]:
# VERIFICANDO TABELAS BANCO DE DADOS
query_table = """
    SELECT tablename
    FROM pg_tables
    WHERE schemaname='pa004'
"""

cur.execute( query_table )

record = cur.fetchall()
print(record)

#### Colecting Datas from DataBase using Pandas

In [ ]:
query_all_datas = """
    SELECT *
    FROM pa004.users u
    INNER JOIN pa004.vehicle v ON ( u.id = v.id )
    INNER JOIN pa004.insurance i ON (u.id = i.id )
"""
df_raw = pd.read_sql( query_all_datas, conn )
cur.close()
conn.close()

## 1- DATA DESCRIPTION

In [ ]:
df1 = df_raw.copy()

### 1.1- Size of Dataset

In [ ]:
df1.shape

### 1.2- Check Data Types

In [ ]:
df1.dtypes

### 1.3- Check NA Datas

In [ ]:
df1.isna().sum()

In [ ]:
df1.shape

### 1.4- Removing Duplicated Columns

In [ ]:
df1.head()

In [ ]:
df1 = df1.loc[ :,~df1.columns.duplicated() ]

In [ ]:
df1.shape

### 1.5- Descriptive Statisctical

In [3]:
df1 = pd.read_csv('/home/reng/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/data/data.csv')

In [7]:
df1['vehicle_age'].unique()

array(['< 1 Year', '1-2 Year', '> 2 Years'], dtype=object)

In [4]:
num_attributes = df1.select_dtypes( include=[ 'int32', 'int64', 'float64' ] )
cat_attributes = df1.select_dtypes( exclude=[ 'object'] )

In [ ]:
num_attributes

In [5]:
# Central Tendency - mean, median
ct1 = pd.DataFrame( num_attributes.apply( np.mean ) ).T
ct2 = pd.DataFrame( num_attributes.apply( np.median ) ).T

# Dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame( num_attributes.apply( np.std ) ).T
d2 = pd.DataFrame( num_attributes.apply( min ) ).T
d3 = pd.DataFrame( num_attributes.apply( max ) ).T
d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() ) ).T
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() ) ).T
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() ) ).T

# Concatenar
m = pd.concat( [ d2, d3, d4, ct1, ct2, d1, d5, d6 ] ).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,Unnamed: 0,0.0,381108.0,381108.0,190554.000000,190554.0,110016.691870,9.443274e-16,-1.200000
1,id,1.0,381109.0,381108.0,190555.000000,190555.0,110016.691870,-8.071174e-18,-1.200000
2,age,20.0,85.0,65.0,38.822584,36.0,15.511591,6.725390e-01,-0.565655
3,region_code,0.0,52.0,52.0,26.388807,28.0,13.229871,-1.152664e-01,-0.867857
4,policy_sales_channel,1.0,163.0,162.0,112.034295,133.0,54.203924,-9.000081e-01,-0.970810
5,driving_license,0.0,1.0,1.0,0.997869,1.0,0.046109,-2.159518e+01,464.354302
6,previously_insured,0.0,1.0,1.0,0.458210,0.0,0.498251,1.677471e-01,-1.971871
7,annual_premium,2630.0,540165.0,537535.0,30564.389581,31669.0,17213.132474,1.766087e+00,34.004569
8,vintage,10.0,299.0,289.0,154.347397,154.0,83.671194,3.029517e-03,-1.200688
9,response,0.0,1.0,1.0,0.122563,0.0,0.327935,2.301906e+00,3.298788


## 2- FEATURE ENGINEERING

In [ ]:
df2 = df1.copy()

In [ ]:
# Salvando os dados coletados em csv na minha máquina
os.makedirs('/home/reng/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/data', exist_ok=True) #Criando pasta
df2.to_csv('/home/reng/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/data/data.csv')# Salvando os dados coletados em csv na minha máquina

In [ ]:
df2 = pd.read_csv('/home/reng/Documents/ds_repos/projects/Health_Insurance_Cross_Sell/data/data.csv')

In [ ]:
df2.head()